In [13]:
import tensorflow as tf
from keras import layers, models, callbacks
import numpy as np
import matplotlib.pyplot as plt


In [14]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1,28,28,1).astype('float32') / 255.0
x_test  = x_test.reshape(-1,28,28,1).astype('float32') / 255.0

y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test_oh  = tf.keras.utils.to_categorical(y_test,  num_classes=10)


In [ ]:
def build_cnn():
    model = tf.keras.Sequential([
        layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),  

        layers.Dense(10, activation='softmax')
    ])
    return model


In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)
lr_schedule1 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
lr_schedule2 = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.01 * 0.95**epoch, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.15)


model = build_cnn()
model.compile(optimizer=optimizer, loss=loss_fn , metrics=['accuracy'])

model.fit(x_train, y_train_oh, epochs=50, batch_size=256, validation_split=0.1, callbacks=[lr_schedule2, early_stop])



Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/50
211/211 [==============================] - 5s 19ms/step - loss: 0.9623 - accuracy: 0.9329 - val_loss: 3.1037 - val_accuracy: 0.1050 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.0095.
Epoch 2/50
211/211 [==============================] - 4s 17ms/step - loss: 0.8126 - accuracy: 0.9806 - val_loss: 1.1953 - val_accuracy: 0.8562 - lr: 0.0095

Epoch 3: LearningRateScheduler setting learning rate to 0.009025.
Epoch 3/50
211/211 [==============================] - 4s 18ms/step - loss: 0.7920 - accuracy: 0.9864 - val_loss: 0.7736 - val_accuracy: 0.9888 - lr: 0.0090

Epoch 4: LearningRateScheduler setting learning rate to 0.00857375.
Epoch 4/50
211/211 [==============================] - 4s 19ms/step - loss: 0.7816 - accuracy: 0.9889 - val_loss: 0.7431 - val_accuracy: 0.9902 - lr: 0.0086

Epoch 5: LearningRateScheduler setting learning rate to 0.0081450625.
Epoch 5/50
211/211 [=====================

In [22]:
test_loss, test_acc = model.evaluate(x_test, y_test_oh)
print(f"Test accuracy: {test_acc*100:.2f}")


313/313 [==============================] - 2s 5ms/step - loss: 0.7234 - accuracy: 0.9934
Test accuracy: 99.34
